In [56]:
import gradio as gr
import torch
import torchtext
from torchtext import data
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [20]:
glove = torchtext.vocab.GloVe(name="6B",dim=100)
embedding_dim = 100
num_filters = 50
k_size=[2,4]
freeze = True

In [102]:
class BaselineModel(torch.nn.Module):
    def __init__(self, vocab, embedding_dim):
        super(BaselineModel,self).__init__()
        self.embedding = nn.Embedding.from_pretrained(vocab.vectors)
        self.linear = nn.Linear(embedding_dim,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        """
        x: torch.tensor of shape (bsz), bsz is the batch size
        """
        out = self.embedding(x)
        out = torch.mean(out,0)
        logits = self.linear(out).squeeze(1)
        logits = self.sigmoid(logits)
        return logits

In [103]:
class CNNmodel(torch.nn.Module):
    def __init__(self,vocab,embedding_dim,num_filters,k_size, freeze):
        super(CNNmodel,self).__init__()
        self.embedding = nn.Embedding.from_pretrained(vocab.vectors, freeze=freeze)
        self.conv1 = nn.Conv2d(1,num_filters,kernel_size=(k_size[0],embedding_dim),bias=False)
        self.conv2 = nn.Conv2d(1,num_filters,kernel_size=(k_size[1],embedding_dim),bias=False)
        self.linear1 = nn.Linear(num_filters*2,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x, length,k_size):
        """
        x: torch.tensor of shape (bsz), bsz is the batch size
        """
        out = self.embedding(x)
        out = out.permute(1,0,2).unsqueeze(1)
        out1 = self.conv1(out).squeeze(3)
        out1 = F.relu(out1)
        out1 = F.max_pool1d(out1,int(length-(k_size[0]-1)))
        out2 = self.conv2(out).squeeze(3)
        out2 = F.relu(out2)
        out2 = F.max_pool1d(out2,int(length-(k_size[1]-1)))
        out = torch.concat((out1,out2),1).squeeze()
        logits = self.linear1(out)
        logits = self.sigmoid(logits)
        return logits  

In [105]:
def classification(Text):
    baseline_checkpoint = torch.load('baseline.pt',map_location=torch.device('cpu'))
    baseline_model = BaselineModel(glove,embedding_dim)
    baseline_model.load_state_dict(baseline_checkpoint)
    
    CNN_checkpoint = torch.load('cnn.pt',map_location=torch.device('cpu'))
    CNN_model = CNNmodel(glove,embedding_dim,num_filters,k_size, freeze)
    CNN_model.load_state_dict(CNN_checkpoint)
    
    tokens = Text.split()
    token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
    token_tensor = torch.LongTensor(token_ints).view(-1,1)
    
    baseline_prob = baseline_model(token_tensor)
    baseline_probability = float(baseline_prob)
    cnn_prob = CNN_model(token_tensor, len(token_tensor), k_size)
    cnn_probability = float(cnn_prob)
    baseline_probability = round(baseline_probability, 2)
    cnn_probability = round(cnn_probability, 2)
    
    if baseline_probability > 0.5:
        a,b = 'Subjective', baseline_probability
    else:
        a,b = 'Objective', baseline_probability
    
    if cnn_probability > 0.5:
        c,d = 'Subjective', cnn_probability
    else:
        c,d = 'Objective', cnn_probability
        
    return a,b,c,d

      
output1 = gr.Textbox(label="Baseline classification")
output2 = gr.Textbox(label="Baseline prediction probability")
output3 = gr.Textbox(label="CNN classification")
output4 = gr.Textbox(label="CNN prediction probability")
    
demo = gr.Interface(
    fn=classification,
    inputs=["text"],
    outputs=[output1, output2, output3, output4],
)


demo.launch()

Running on local URL:  http://127.0.0.1:7896

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x1cd0ae7c508>, 'http://127.0.0.1:7896/', None)